In [2]:
import copy
import os

import polib

# Monkey translate strings to know where they can be translated, and where not.

In [53]:
# pofile = polib.pofile('./LC_MESSAGES/crawl.po')

# for entry in pofile.untranslated_entries():
# #     if entry.msgstr.startswith('***') and entry.msgstr.endswith('***'):
#         entry.msgstr = f"^^^{entry.msgid}$$$"
    
# pofile.save()

# Add all monsters to appropriate contexts for correct translation

In [3]:
os.system('cd ../../../; xgettext -k_:1 -k__:1c,2 -o ./lang/po/mon-data.po ./mon-data.h')
monsters_pofile = polib.pofile('../../po/mon-data.po')

ARTICLE_A = "a"
ARTICLE_THE = "the "  # note the trailing space

monsters_contexts = [
    ("", ARTICLE_A),
    ("slain by", ARTICLE_A),
    ("The mighty blow completely misses %s!", ""),
    ("Your bite misses %s.", ARTICLE_THE),
    ("%s hits a putrid demon", ARTICLE_THE),
    ("The giant cockroach hits %s", ARTICLE_A),
]
vowels = 'aeiou'


def new_entries_gen():
    for entry in monsters_pofile:
        if not entry.msgid:
            continue
        for context, article in monsters_contexts:
            new_entry = copy.copy(entry)
            
            if context:
                new_entry.msgctxt = context
            
            if article == ARTICLE_A:
                if new_entry.msgid[0] in vowels:
                    article = 'an '
                else:
                    article = 'a '

            new_entry.msgid = f'{article}{new_entry.msgid}'
            yield new_entry


for entry in list(new_entries_gen()):  # without list we get infinite loop, because we add to list we're iterating on
    monsters_pofile.append(entry)
    
monsters_pofile.save()

# Add all features "a " appended version

In [10]:
os.system('cd ..; xgettext -k_:1 -k__:1c,2 -o ./lang/po/feature-data.po ./feature-data.h')
features_pofile = polib.pofile('./po/feature-data.po')

vowels = 'aeiou'


def new_features_entries_gen():
    for entry in features_pofile:
        if not entry.msgid:
            continue
        new_entry = copy.copy(entry)
        if new_entry.msgid[0] in vowels:
            article = 'an'
        else:
            article = 'a'
        new_entry.msgid = f'{article} {new_entry.msgid}'
        yield new_entry


for entry in list(new_features_entries_gen()):  # without list we get infinite loop, because we add to list we're iterating on
    features_pofile.append(entry)
    
features_pofile.save()

# Вывести список существ

In [21]:
import pandas as pd


os.system('cd ../../../; xgettext -k_:1 -k__:1c,2 -o ./lang/po/mon-data.po ./mon-data.h')
monsters_pofile = polib.pofile('../../po/mon-data.po')
translations_pofile = polib.pofile('../../po/ru/LC_MESSAGES/crawl.po')


def get_monster_ids(monsters_pofile):
    for monster_entry in monsters_pofile:
        yield(monster_entry.msgid)
        

def get_monster_ids_and_translations(monster_ids, translations_pofile):
    data = dict(map(lambda mid: (mid, ""), monster_ids))
    
    for entry in translations_pofile:
        msgid = entry.msgid
        if msgid in data:
            if entry.msgctxt:
                print(f"Skipping ({msgid}): txn in context [{entry.msgctxt}] == <{entry.msgstr}>.")
                continue
            data[msgid] = entry.msgstr
    
    return data
    

df = pd.DataFrame.from_records(
    list(
        get_monster_ids_and_translations(
            get_monster_ids(
                monsters_pofile
            ),
            translations_pofile,
        ).items()
        )
)

Skipping (yak): txn in context [slain by] == <яком>.
Skipping (giant cockroach): txn in context [The mighty blow completely misses %s!] == <гигантскому таракану>.
Skipping (jackal): txn in context [%s bites you for 10 damage with +3 dagger!!!] == <шакал>.
Skipping (jackal): txn in context [The jackal bites %s for 10 damage with +3 dagger!!!] == <шакала>.


In [22]:
df.to_clipboard()